In [ ]:
from langchain import OpenAI, PromptTemplate


from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

import os

from dotenv import load_dotenv


load_dotenv()



# Entity memory

In [ ]:
input_memory = [{'step': 1,
  'user_message': 'Estou com alguns problemas com meu seguro.',
  'bot_response': 'Olá! Bem-vindo ao nosso suporte. Como posso ajudar você hoje?'},
 {'step': 2,
  'user_message': 'Estou com alguns problemas com meu seguro.',
  'bot_response': 'Entendo que você está buscando ajuda para automatizar um processo de atendimento ao cliente no front office. Podemos orientar você nessa questão.'}]

In [ ]:
def extract_context(memory, context):
    for item in context:
        # Load memory variables and save context for the standardized message
        _input = {"input": item["user_message"]}
        memory.load_memory_variables(_input)
        memory.save_context(_input, {"output": item["bot_response"]})

    return memory

In [ ]:
llm = OpenAI(
    temperature=0.5,
    model="text-davinci-003",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    max_tokens=400,
)

template = """Você é um chatbot conversando com um humano sobre seguros e créditos. 
    Por favor, responda em Português BR e use no máximo 2 linhas. Se o humano quiser falar com algum atendente
    por favor, diga que "Um atendente entrarar em contato em breve, aguarde!".

Context: {entities}
Current conversation: {history}
Humano: {input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["entities", "history", "input"], template=template
)

memory = ConversationEntityMemory(llm=llm)

memory = extract_context(memory, input_memory)


conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=prompt,
    memory=memory,
)


In [ ]:
conversation.predict(input="Quero falar com atendente")

In [ ]:
# conversation.predict(input="Ok, vou tentar, quais documentos preciso?")

In [ ]:
memory